# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import pathlib
import sys

ROOT = pathlib.Path().resolve().parent

sys.path.insert(0, str(ROOT))

df = pd.read_csv(ROOT / 'data/dataset.csv', sep='\t')

## Transformers

Есть пример похожей работы на transformers [ссылка](https://huggingface.co/shahp7575/gpt2-horoscopes)


**Для работы модели текстовой генерации необходимо несколько вещей:**

1. Предобученная модель на массиве текстов
2. Адекватный векторный промт - то, что модель будет продолжать

В качестве фичеров для такой маски мы предлагаем использовать следующий подход:

`<Категория> {Категория} <День недели>`


In [4]:
import os
from horoscoper.config import settings
import subprocess
import h5py
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from transformers import GPT2Tokenizer

# load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(settings.MODEL_NAME)

In [5]:
def make_prompt(category, sign):
    return f"<категория> {category} <знак> {sign}"

def make_full_prompt(category, sign, text):
    return make_prompt(category, sign) + f" <гороскоп>: {text}"

In [6]:
make_full_prompt("Общее", "Рыбы", "")

'<категория> Общее <знак> Рыбы <гороскоп>: '

In [24]:
df['features'] = df[['category_ru', 'sing_ru']].apply(lambda x: make_prompt(x[0], x[1]), axis=1)
df

,category_ru,sing_ru,date,month,day,weekday,text_ru,features
0,общее,овен,2013-01-12,1,12,5,"Вы не из тех, кто перестраховывается, и даже ...",<категория> общее <знак> овен
1,общее,овен,2013-01-12,1,12,5,"Не бывает чего-то даром, и если вы не совсем ...",<категория> общее <знак> овен
2,общее,овен,2013-01-12,1,12,5,Когда новолуние попадает в одну из самых аван...,<категория> общее <знак> овен
3,общее,овен,2013-01-12,1,12,5,"Сегодня вы услышите нечто удивительное, но мо...",<категория> общее <знак> овен
4,общее,овен,2013-01-12,1,12,5,"Друг или коллега, которого вы давно не видели,...",<категория> общее <знак> овен
...,...,...,...,...,...,...,...,...
34916,др,рыбы,2021-01-12,1,12,1,"Стильно отпразднуйте свой день рождения, чтобы...",<категория> др <знак> рыбы
34917,др,рыбы,2021-01-13,1,13,2,"Представьте свою жизнь так, как если бы она бы...",<категория> др <знак> рыбы
34918,др,рыбы,2021-01-14,1,14,3,"Веселье, игривость и юмор легко проявятся в эт...",<категория> др <знак> рыбы
34919,др,рыбы,2021-01-15,1,15,4,"Ваш день рождения дает вам новое начало, когда...",<категория> др <знак> рыбы


In [27]:
df['full'] = df[['category_ru', 'sing_ru', 'text_ru']].apply(lambda x: make_full_prompt(x[0], x[1], x[2]), axis=1)
df

,category_ru,sing_ru,date,month,day,weekday,text_ru,features,full
0,общее,овен,2013-01-12,1,12,5,"Вы не из тех, кто перестраховывается, и даже ...",<категория> общее <знак> овен,<категория> общее <знак> овен <гороскоп>: Вы ...
1,общее,овен,2013-01-12,1,12,5,"Не бывает чего-то даром, и если вы не совсем ...",<категория> общее <знак> овен,<категория> общее <знак> овен <гороскоп>: Не ...
2,общее,овен,2013-01-12,1,12,5,Когда новолуние попадает в одну из самых аван...,<категория> общее <знак> овен,<категория> общее <знак> овен <гороскоп>: Ког...
3,общее,овен,2013-01-12,1,12,5,"Сегодня вы услышите нечто удивительное, но мо...",<категория> общее <знак> овен,<категория> общее <знак> овен <гороскоп>: Сег...
4,общее,овен,2013-01-12,1,12,5,"Друг или коллега, которого вы давно не видели,...",<категория> общее <знак> овен,<категория> общее <знак> овен <гороскоп>: Друг...
...,...,...,...,...,...,...,...,...,...
34916,др,рыбы,2021-01-12,1,12,1,"Стильно отпразднуйте свой день рождения, чтобы...",<категория> др <знак> рыбы,<категория> др <знак> рыбы <гороскоп>: Стильно...
34917,др,рыбы,2021-01-13,1,13,2,"Представьте свою жизнь так, как если бы она бы...",<категория> др <знак> рыбы,<категория> др <знак> рыбы <гороскоп>: Предста...
34918,др,рыбы,2021-01-14,1,14,3,"Веселье, игривость и юмор легко проявятся в эт...",<категория> др <знак> рыбы,<категория> др <знак> рыбы <гороскоп>: Веселье...
34919,др,рыбы,2021-01-15,1,15,4,"Ваш день рождения дает вам новое начало, когда...",<категория> др <знак> рыбы,<категория> др <знак> рыбы <гороскоп>: Ваш ден...


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, random_state=42, stratify=df['features'], test_size=0.25)

In [32]:
train_texts = X_train['full']
valid_texts = X_test['full']

In [33]:
with open('train.txt', 'w') as f:
    for t in train_texts:
        f.write(t + '\n')

In [34]:
with open('valid.txt', 'w') as f:
    for t in valid_texts:
        f.write(t + '\n')